# SDP HPSO Scheduling

Last run with Jupyter Notebook 5.0.0 running Python 3.5.2

In [ ]:
# Imports
import matplotlib.pyplot as plt
import sys
import os
import pickle

sys.path += ['..']
from sdp_par_model import reports as iapi
from sdp_par_model.parameters.definitions import *
from sdp_par_model.parameters.definitions import Constants as c

from sdp_par_model.scheduler import Definitions as sdefs
from sdp_par_model.scheduler import Scheduler

import collections
import warnings
import bisect

%matplotlib inline
plt.rcParams['figure.figsize'] = 16, 8

## Create a Scheduler object
At the moment the Scheduler object contains the functionality of the SDP Simulator, the scheduler, as well as the generated schedule. In future we may wish to split them into separate classes and objects 

In [ ]:
sdp_scheduler = Scheduler()

## Read  performace requirement lookup for all HPSOs. 
### If this lookup table does not exist, we create it, and save it to disk (to save time re-computing)

In [ ]:
performance_lookup_filename = "performance_dict.data"
if os.path.isfile(performance_lookup_filename):
    performance_dict = None
    with open(performance_lookup_filename, "rb") as f:
        performance_dict = pickle.load(f)
    sdp_scheduler.set_performance_dictionary(performance_dict)
else:
    # Create a performance dictionary and write it to file
    performance_dict = sdp_scheduler.compute_performance_dictionary()
    with open(performance_lookup_filename, "wb") as f:
        pickle.dump(performance_dict, f, pickle.HIGHEST_PROTOCOL)

## Let's create run a short test sequence
### This can be modified so that the list is read from e.g. a CSV file

In [ ]:
seqL = ('A','A','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','A','A','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','A','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B')
seqM = ('B','G','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','G','C','F','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','G','G','E','E','E','E','D')

In [ ]:
''' To show how the tasks are created, we convert a test sequence to Task objects.
 (The two A's at the start isn't a mistake -- it shows how a repeat of the same task type leads to unique 
 subtask IDs being created '''

test_seq = ('A','A','B','C','E','F','G')
test_seq = ('A',)

sdp_scheduler.task_letters_to_sdp_task_list(test_seq)
for task in sdp_scheduler.task_list:
    print(task)

In [ ]:
sdp_scheduler.schedule(sdp_flops=22.8, assign_flops_fraction=0.3, assign_bandwidth_fraction=0.8, max_nr_iterations=10)

In [ ]:
iapi.plot_deltas(sdp_scheduler.flops_deltas, 'Evolution of SDP FLOP/s', 'wall clock time (hours)', 'PetaFLOP/s')
iapi.plot_deltas(sdp_scheduler.memory_deltas, 'Evolution of SDP working memory (RAM)', 'wall clock time (hours)', 'TeraByte')
iapi.plot_deltas(sdp_scheduler.cold_buffer_deltas, 'Evolution of SDP Cold buffer', 'wall clock time (hours)', 'TeraByte')
iapi.plot_deltas(sdp_scheduler.hot_buffer_deltas, 'Evolution of SDP Hot buffer', 'wall clock time (hours)', 'TeraByte')
iapi.plot_deltas(sdp_scheduler.preserve_deltas, 'Evolution of SDP Preservation storage', 'wall clock time (hours)', 'TeraByte')

# The blocks below this line are legacy (test) code.
## It will probably not be able to run due to changes to the code since they were written. For reference use only! (advice: only execute if you think that know what you're doing)

## Simulate the execution of this sequence on the SDP and plot the results

In [ ]:
sequence_to_simulate = seqL

task_list = sdp_scheduler.task_letters_to_sdp_task_list(sequence_to_simulate)

schedule = Scheduler.schedule(task_list, sdp_FLOPS, max_nr_iterations=1000, assign_sdp_fraction=0.3)

## Hard-coded performace costs and requirements from Rosie's Excel sheet
### These were previously used in rev [3372fdd] to approximately replicate Rosie's results. Check (rerun) the notebook at that repository revision to regenerate those results - not repeated here.

In [ ]:
# The following sets of values should be computed using the parametric model. Just hard-coded for now (from Excel)
hpso_ingest_rates = {'A':0.459, 'B':3e-3, 'C':0.117, 'D':0.112, 'E':0.0603, 'F':0.244, 'G':0.438}  # in TeraByte/s
# FLOPcounts below are the PetaFLOPs required to process one second of ingested data
hpso_flopcounts = {'A':50.4, 'B':2.0, 'C':7.5, 'D':6.2, 'E':2.9833, 'F':17.689, 'G':27.698}  # in PetaFLOP/s
hpso_durations  = {'A':6, 'B':0.17, 'C':6, 'D':6, 'E':4.4, 'F':0.1233, 'G':6}  # in hours -- TODO check whether correct

sdp_setup_time = 60  # the minimum amount of time between processing tasks on the SDP (seconds)
telecope_setup_time = 0  # TODO is this correct?

## Reproduction of "Low" and "Mid" sequences from Rosie's Excel sheet
### Create a lists of observation tasks as letter sequences

In [ ]:
seqL = ('A','A','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','A','A','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','A','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B')
seqM = ('B','G','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','G','C','F','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','G','G','E','E','E','E','D')

print('HPSO LOW task distribution (number of occurences) A..B = (%.0f, %.0f)' % (seqL.count('A'), seqL.count('B')))
tA = seqL.count('A') * hpso_durations['A']
tB = seqL.count('B') * hpso_durations['B']
print('HPSO LOW task distribution (observation time) A..B = (%.1f%%, %.1f%%)' % (100 * tA / (tA + tB), 100 * tB / (tA + tB)))

tA = seqM.count('A')
tB = seqM.count('B')
tC = seqM.count('C')
tD = seqM.count('D')
tE = seqM.count('E')
tF = seqM.count('F')
tG = seqM.count('G')
tt = len(seqM)

print('\nHPSO MID task distribution (number of occurences) A..G = (%.0f, %.0f, %.0f, %.0f, %.0f, %.0f, %.0f)' % \
      (tA, tB, tC, tD, tE, tF, tG))
print('HPSO MID task distribution (observation time) A..G = (%.1f%%, %.1f%%, %.1f%%, %.1f%%, %.1f%%, %.1f%%, %.1f%%)' % \
      (100*tA/tt, 100*tB/tt, 100*tC/tt, 100*tD/tt, 100*tE/tt, 100*tF/tt, 100*tG/tt))